In [6]:
import os
import cv2
import gradio as gr
from ultralytics import YOLO
import numpy as np

# ==================== 模型加载与推理函数 ====================
class YOLODetector:
    def __init__(self, model_path):
        """初始化模型，加载训练好的YOLO权重"""
        self.model = YOLO(model_path)
        # 类别名称（需与训练时的class_mapping对应）
        self.class_names = ['聚酯电容', '热敏电阻', '三端稳压管']
        
        print(f"模型加载成功：{model_path}")

    def detect(self, image, conf_threshold=0.5):
        """
        执行目标检测
        参数：
            image: 输入图片（Gradio会自动转换为numpy数组）
            conf_threshold: 置信度阈值
        返回：
            result_img: 带检测框的图片
            detections: 检测结果文本（类别+置信度）
        """
        # 转换图片格式（Gradio默认RGB，OpenCV默认BGR，需统一）
        img_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 模型推理
        results = self.model(img_bgr, conf=conf_threshold)
        
        # 绘制检测框并提取结果
        result_img = img_bgr  # 初始化为原图
        detections = []
        
        for result in results:
            for box in result.boxes:
                # 获取坐标（转换为整数）
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                # 获取类别和置信度
                cls_id = int(box.cls[0])
                cls_name = self.class_names[cls_id]
                conf = float(box.conf[0])
                
                # 绘制检测框和标签（绿色框+白色文字）
                cv2.rectangle(result_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(
                    result_img,
                    f"{cls_name} {conf:.2f}",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (255, 255, 255), 2
                )
                
                # 记录检测结果
                detections.append(f"- {cls_name}（置信度：{conf:.2f}）")
        
        # 转换回RGB格式（Gradio显示需要）
        result_img_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
        
        # 格式化检测结果文本
        if detections:
            detection_text = "检测到以下目标：\n" + "\n".join(detections)
        else:
            detection_text = "未检测到任何目标"
        
        return result_img_rgb, detection_text

# ==================== 初始化模型 ====================
# 替换为你的模型路径（训练好的best.pt）
MODEL_PATH = "./runs/detect/train2/weights/best.pt"
detector = YOLODetector(MODEL_PATH)

# ==================== 定义Gradio交互函数 ====================
def inference_interface(image, conf_threshold):
    """Gradio界面的核心交互函数"""
    if image is None:
        return None, "请先上传图片"
    # 调用检测函数
    result_img, detection_text = detector.detect(image, conf_threshold)
    return result_img, detection_text

# ==================== 构建Gradio界面 ====================
with gr.Blocks(title="YOLO目标检测工具") as demo:
    gr.Markdown("# 📌 YOLO目标检测推理界面")
    gr.Markdown("上传图片，调整置信度阈值，点击检测按钮查看结果（支持聚酯电容、热敏电阻、三端稳压管检测）")
    
    # 布局：左侧输入，右侧输出
    with gr.Row():
        # 左侧：输入区域
        with gr.Column(scale=1):
            input_image = gr.Image(
                type="numpy", 
                label="上传图片",
                height=300
            )
            conf_slider = gr.Slider(
                minimum=0.1,
                maximum=1.0,
                value=0.5,
                step=0.05,
                label="置信度阈值（数值越低，检测越灵敏）"
            )
            detect_btn = gr.Button("开始检测", variant="primary")
        
        # 右侧：输出区域
        with gr.Column(scale=1):
            output_image = gr.Image(
                type="numpy", 
                label="检测结果（带边框）",
                height=300
            )
            output_text = gr.Textbox(
                label="检测结果详情",
                lines=6,
                placeholder="检测结果将显示在这里..."
            )
    
    # 绑定交互逻辑：点击按钮触发检测
    detect_btn.click(
        fn=inference_interface,
        inputs=[input_image, conf_slider],
        outputs=[output_image, output_text]
    )
    
    # 额外：支持上传图片后自动检测（可选）
    input_image.change(
        fn=inference_interface,
        inputs=[input_image, conf_slider],
        outputs=[output_image, output_text]
    )

# ==================== 运行界面 ====================
if __name__ == "__main__":
    # 启动Gradio服务，server_name="0.0.0.0"允许局域网访问
    demo.launch(server_name="0.0.0.0", server_port=7862, share=False)

模型加载成功：./runs/detect/train2/weights/best.pt
* Running on local URL:  http://0.0.0.0:7862
* To create a public link, set `share=True` in `launch()`.



0: 480x640 2 聚酯电容s, 1 热敏电阻, 2 三端稳压管s, 121.5ms
Speed: 2.8ms preprocess, 121.5ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)


In [8]:
import os
import cv2
import gradio as gr
from ultralytics import YOLO
import numpy as np

# ==================== 模型加载与推理函数 ====================
class YOLODetector:
    def __init__(self, model_path):
        """初始化模型，加载训练好的YOLO权重"""
        self.model = YOLO(model_path)
        # 类别名称（需与训练时的class_mapping对应）
        self.class_names = ['聚酯电容', '热敏电阻', '三端稳压管']
        # 定义类别与颜色的映射，这里使用RGB颜色值，你可以根据实际训练时的颜色调整
        self.class_colors = {
            '聚酯电容': (0, 0, 255),  # 红色
            '热敏电阻': (0, 255, 0),  # 绿色
            '三端稳压管': (0, 0, 0)  # 蓝色
        }
        print(f"模型加载成功：{model_path}")

    def detect(self, image, conf_threshold=0.5):
        """
        执行目标检测
        参数：
            image: 输入图片（Gradio会自动转换为numpy数组）
            conf_threshold: 置信度阈值
        返回：
            result_img: 带检测框的图片
            detections: 检测结果文本（类别+置信度）
        """
        # 转换图片格式（Gradio默认RGB，OpenCV默认BGR，需统一）
        img_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # 模型推理
        results = self.model(img_bgr, conf=conf_threshold)
        
        # 绘制检测框并提取结果
        result_img = img_bgr  # 初始化为原图
        detections = []
        
        for result in results:
            for box in result.boxes:
                # 获取坐标（转换为整数）
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                # 获取类别和置信度
                cls_id = int(box.cls[0])
                cls_name = self.class_names[cls_id]
                conf = float(box.conf[0])
                
                # 根据类别获取对应的颜色
                color = self.class_colors[cls_name]
                
                # 绘制检测框和标签（使用获取到的颜色框+白色文字）
                cv2.rectangle(result_img, (x1, y1), (x2, y2), color, 2)
                cv2.putText(
                    result_img,
                    f"{cls_name} {conf:.2f}",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (255, 255, 255), 2
                )
                
                # 记录检测结果
                detections.append(f"- {cls_name}（置信度：{conf:.2f}）")
        
        # 转换回RGB格式（Gradio显示需要）
        result_img_rgb = cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)
        
        # 格式化检测结果文本
        if detections:
            detection_text = "检测到以下目标：\n" + "\n".join(detections)
        else:
            detection_text = "未检测到任何目标"
        
        return result_img_rgb, detection_text

# ==================== 初始化模型 ====================
# 替换为你的模型路径（训练好的best.pt）
MODEL_PATH = "./runs/detect/train2/weights/best.pt"
detector = YOLODetector(MODEL_PATH)

# ==================== 定义Gradio交互函数 ====================
def inference_interface(image, conf_threshold):
    """Gradio界面的核心交互函数"""
    if image is None:
        return None, "请先上传图片"
    # 调用检测函数
    result_img, detection_text = detector.detect(image, conf_threshold)
    return result_img, detection_text

# ==================== 构建Gradio界面 ====================
with gr.Blocks(title="YOLO目标检测工具") as demo:
    gr.Markdown("# 📌 YOLO目标检测推理界面")
    gr.Markdown("上传图片，调整置信度阈值，点击检测按钮查看结果（支持聚酯电容、热敏电阻、三端稳压管检测）")
    
    # 布局：左侧输入，右侧输出
    with gr.Row():
        # 左侧：输入区域
        with gr.Column(scale=1):
            input_image = gr.Image(
                type="numpy", 
                label="上传图片",
                height=300
            )
            conf_slider = gr.Slider(
                minimum=0.1,
                maximum=1.0,
                value=0.5,
                step=0.05,
                label="置信度阈值（数值越低，检测越灵敏）"
            )
            detect_btn = gr.Button("开始检测", variant="primary")
        
        # 右侧：输出区域
        with gr.Column(scale=1):
            output_image = gr.Image(
                type="numpy", 
                label="检测结果（带边框）",
                height=300
            )
            output_text = gr.Textbox(
                label="检测结果详情",
                lines=6,
                placeholder="检测结果将显示在这里..."
            )
    
    # 绑定交互逻辑：点击按钮触发检测
    detect_btn.click(
        fn=inference_interface,
        inputs=[input_image, conf_slider],
        outputs=[output_image, output_text]
    )
    
    # 额外：支持上传图片后自动检测（可选）
    input_image.change(
        fn=inference_interface,
        inputs=[input_image, conf_slider],
        outputs=[output_image, output_text]
    )

# ==================== 运行界面 ====================
if __name__ == "__main__":
    # 启动Gradio服务，server_name="0.0.0.0"允许局域网访问
    demo.launch(server_name="0.0.0.0", server_port=10085, share=False)

模型加载成功：./runs/detect/train2/weights/best.pt
* Running on local URL:  http://0.0.0.0:10085
* To create a public link, set `share=True` in `launch()`.



0: 480x640 1 热敏电阻, 118.1ms
Speed: 3.0ms preprocess, 118.1ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 6.9ms
Speed: 2.8ms preprocess, 6.9ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 聚酯电容, 1 热敏电阻, 6.9ms
Speed: 2.4ms preprocess, 6.9ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 热敏电阻, 4 三端稳压管s, 126.9ms
Speed: 3.1ms preprocess, 126.9ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 热敏电阻, 4 三端稳压管s, 6.9ms
Speed: 3.4ms preprocess, 6.9ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 聚酯电容, 1 热敏电阻, 4 三端稳压管s, 6.8ms
Speed: 2.3ms preprocess, 6.8ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 聚酯电容, 1 热敏电阻, 4 三端稳压管s, 7.0ms
Speed: 2.2ms preprocess, 7.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 聚酯电容, 1 热敏电阻, 6 三端稳压管s, 6.9ms
Speed: 2.3ms preproce